## Import Modules

In [ ]:
%load_ext autoreload
%autoreload 2

from ble import get_ble_controller
from base_ble import LOG
from cmd_types import CMD
from ble_rx_stream import *
import time
import numpy as np
import matplotlib.pyplot as plt

import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display
%matplotlib inline

LOG.propagate = False

## Connect to Artemis

In [ ]:
ble = get_ble_controller()
ble.connect()

## Run Map

In [ ]:
calibrate_s = 1.5
max_angle = 360.0
delta_angle = 20.0
tolerance = 1.0
min_pid = tolerance
max_pid = delta_angle + 2 * tolerance
min_motor = 120
max_motor = 180
k_p = -1.0
k_i = -0.5
k_d = 0.0
ble.send_command(CMD.RUN_MAP, f"{int(calibrate_s*1000)}|{max_angle}|{delta_angle}|{tolerance}|{min_pid}|{max_pid}|{min_motor}|{max_motor}|{k_p}|{k_i}|{k_d}")

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.DATA_MAP, "")
stream_map = list(unpack_stream('LfH*', await fut))

In [ ]:
fut = ble_rx_stream(ble, 'RX_STREAM')
ble.send_command(CMD.PID_MAP, "")
stream_pid = list(unpack_stream('LfffffL*', await fut))

In [ ]:
map_data = np.array(stream_map)
map_time = map_data[:, 0] / 1000
map_angl = map_data[:, 1]
map_dist = map_data[:, 2]

In [ ]:
pid_data = np.array(stream_pid)
pid_time = pid_data[:, 0] / 1000
pid_sped = pid_data[:, 1]
pid_angl = pid_data[:, 2]
pid_errr = pid_data[:, 3]
pid_pidv = pid_data[:, 4]
pid_motr = pid_data[:, 5]

## Plot Data

In [ ]:
data_dict = {}

In [ ]:
map_data = np.array(stream_map)
map_time = map_data[:, 0] / 1000
map_angl = map_data[:, 1]
map_dist = map_data[:, 2]
data_dict[0, 0] = (map_time, map_angl, map_dist, 1.00)

In [ ]:
@widgets.interact(scale=(1.0, 1.5, 0.01))
def do_polar_plot(scale=1.25):
    plt.polar(np.radians(map_angl * scale), map_dist)
    plt.savefig('polar_00_scl.jpg')
    plt.show()

In [ ]:
walls = [
    (-1.6764,0.1524), (-1.6764,-1.3716),
    (-1.6764,-1.3716), (1.9812,-1.3716),
    (1.9812,-1.3716), (1.9812,1.3716),
    (1.9812,1.3716), (-0.7620,1.3716),
    (-0.7620,1.3716), (-0.7620,0.1524),
    (-0.7620,0.1524), (-1.6764,0.1524),
    (0.7620,-0.1524), (1.3716,-0.1524),
    (1.3716,-0.1524), (1.3716,0.4572),
    (1.3716,0.4572), (0.7620,0.4572),
    (0.7620,0.4572), (0.7620,-0.1524),
    (-0.1524,-1.3716), (-0.1524,-0.7620),
    (-0.1524,-0.7620), (0.1524,-0.7620),
    (0.1524,-0.7620), (0.1524,-1.3716),
]
starts = [
    (1895, 1331),
    (1895, -1411),
    (-1572, -1315),
    (-1435, 527),
    (-608, -36),
    (-222, 1331),
    (-80, -1355),
]
ends = [
    (1895, -1411),
    (-1572, -1315),
    (-1435, 527),
    (-608, -36),
    (-222, 1331),
    (1895, 1331),
    (-213, -597),
]
@widgets.interact(x=(-2000, 3000), y=(-2000, 3000))
def do_plot_point(x=0, y=0, do_walls=True, do_pointer=False):
    if do_walls:
        for i in range(len(walls) // 2):
            plt.plot([walls[2*i][0] * 1000, walls[2*i+1][0] * 1000], [walls[2*i][1] * 1000, walls[2*i+1][1] * 1000], color='black')
    for i in range(len(starts)):
        plt.plot([starts[i][0], ends[i][0]], [starts[i][1], ends[i][1]], color='magenta')
    if do_pointer:
        plt.plot(x, y, color='magenta', marker='x')
    for (x,y) in data_dict.keys():
        map_time, map_angl, map_dist, scale = data_dict[x,y]
        plt.scatter(x * 304.8 + map_dist * np.cos(np.radians(map_angl * scale)), y * 304.8 + map_dist * np.sin(np.radians(map_angl * scale)), label=f'({x}, {y})')
    plt.legend()
    plt.savefig('room.jpg')
    plt.show()

In [ ]:
_, axs = plt.subplots(2, 1, sharex=False)
axs[0].plot(map_time, map_dist)
axs[0].set_ylabel('distance (mm)')
axs[0].set_xlabel('time (s)')
axs[1].polar(map_angl, map_dist)
plt.show()

In [ ]:
_, axs = plt.subplots(3, 1, sharex=True)
axs[0].plot(pid_time, pid_sped)
axs[0].set_ylabel('angular speed\n(deg/s)')
axs[1].plot(pid_time, pid_angl)
axs[1].axhline(360, c='red', ls='--')
axs[1].set_ylabel('angle\n(deg)')
axs[2].plot(pid_time, pid_motr)
axs[2].set_ylabel('motor output')
plt.xlabel('time (s)')
plt.savefig('pid.jpg')
plt.show()

## Disconnect from Artemis

In [ ]:
ble.disconnect()